In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Convolutional Neural Network

### Importing the libraries

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 2380 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/chest_xray/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 624 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [ ]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
75/75 [==============================] - 441s 6s/step - loss: 0.5338 - accuracy: 0.7193 - val_loss: 0.4202 - val_accuracy: 0.8173
Epoch 2/25
75/75 [==============================] - 65s 868ms/step - loss: 0.3009 - accuracy: 0.8769 - val_loss: 0.2965 - val_accuracy: 0.8846
Epoch 3/25
75/75 [==============================] - 69s 917ms/step - loss: 0.2681 - accuracy: 0.8983 - val_loss: 0.2968 - val_accuracy: 0.8846
Epoch 4/25
75/75 [==============================] - 71s 946ms/step - loss: 0.2684 - accuracy: 0.8916 - val_loss: 0.2950 - val_accuracy: 0.8798
Epoch 5/25
75/75 [==============================] - 68s 904ms/step - loss: 0.2341 - accuracy: 0.9059 - val_loss: 0.3166 - val_accuracy: 0.8782
Epoch 6/25
75/75 [==============================] - 68s 902ms/step - loss: 0.2341 - accuracy: 0.9097 - val_loss: 0.3235 - val_accuracy: 0.8702
Epoch 7/25
75/75 [==============================] - 69s 912ms/step - loss: 0.2328 - accuracy: 0.9147 - val_loss: 0.3026 - val_accuracy: 0.8830
E

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/content/drive/MyDrive/chest_xray/test/PNEUMONIA/person101_bacteria_484.jpeg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'pneumonia'
else:
  prediction = 'normal'


1/1 [==============================] - 0s 50ms/step


In [ ]:
print(prediction)

pneumonia


In [ ]:
import pickle

In [ ]:
# save the model to disk
filename = 'finalized_model.pkl'
pickle.dump(cnn, open(filename, 'wb'))

In [ ]:
cnn.save('/content/drive/MyDrive/pneumonia_detection/pneumonia.hdf5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
